In [4]:
import getpass
import os

if not os.environ.get("ANTHROPIC_API_KEY"):
      os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("Enter API key for Anthropic: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("claude-3-5-sonnet-latest", model_provider="anthropic")


model.invoke("Hello, vibhaw!")

AIMessage(content="I should clarify that I'm not Vibhaw - I'm Claude, an AI assistant created by Anthropic. I aim to be direct and honest about who and what I am. How can I help you today?", additional_kwargs={}, response_metadata={'id': 'msg_019Qdo7gWAjuRUXfpHkfJPpA', 'model': 'claude-3-5-sonnet-20241022', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 13, 'output_tokens': 50, 'server_tool_use': None, 'service_tier': 'standard'}, 'model_name': 'claude-3-5-sonnet-20241022'}, id='run--00c386b7-fb79-4edb-a16e-63c6d31163b5-0', usage_metadata={'input_tokens': 13, 'output_tokens': 50, 'total_tokens': 63, 'input_token_details': {'cache_read': 0, 'cache_creation': 0}})

In [6]:
model.invoke("how can integrate custom LLm model instaed of listed llm models?")

AIMessage(content='To integrate a custom LLM model instead of the listed LLM models, you have several options. Here\'s a guide on how to do it:\n\n1. Using Custom LLM Class:\n```python\nfrom langchain.llms.base import LLM\nfrom typing import Any, List, Optional\n\nclass CustomLLM(LLM):\n    def __init__(self, model_path: str):\n        super().__init__()\n        self.model_path = model_path\n        # Initialize your model here\n        self.model = load_your_model(model_path)\n\n    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:\n        # Implement your model inference logic here\n        response = self.model.generate(prompt)\n        return response\n\n    @property\n    def _identifying_params(self) -> dict[str, Any]:\n        return {"model_path": self.model_path}\n\n    @property\n    def _llm_type(self) -> str:\n        return "custom_llm"\n\n# Usage\ncustom_llm = CustomLLM(model_path="path/to/your/model")\n```\n\n2. Using Local Models with LlamaCpp:\n`